# 期限超過メールのアーカイブ
* Outlook側に検索フォルダ作る="#移動対象"
* 

In [6]:
import win32com.client
import json
import datetime

# 定義
SRC_NAME = "#FY2020"
DEST_NAME = "2020"

# Outlook関係のオブジェクト初期化
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
print("root:", root.name)
stores = app.Session.Stores
dest = None
for store in stores:
    folder = store.GetRootFolder()
    if DEST_NAME in folder.name:
        dest = folder
print("dest", dest.name)

ns = app.GetNamespace("MAPI")
# messages = inbox.Items

src_folder = None
for folder in app.Session.DefaultStore.GetSearchFolders():
    if SRC_NAME in folder.name:
        src_folder = folder
print(src_folder.name)

root: Jiro.Kogi@nttdata.com
dest 2020
#FY2020


In [7]:
def findfolder(root, name):
    for folder in root.folders:
        if folder.folderpath == name:
            return folder
        f = findfolder(folder, name)
        if f != None:
            return f
    return None


def find_sub(parent, name):
    for folder in parent.folders:
        if folder.name == name:
            return folder
    return None


def createFolder(root, sub_name):
    if sub_name.startswith("\\"):
        sub_name = sub_name[1:]
    names = sub_name.split("\\")
    # print(names)
    parent = root
    for name in names:
        f = find_sub(parent, name)
        if f is None:
            parent = parent.folders.add(name)
        else:
            parent = f
    # pint(parent.folderpath)
    return parent


def folder_test():
    test_sub = r"\#12 関西\01 竹中"
    # name = dest.folderpath+"\#05 PMO"
    name = dest.folderpath + test_sub
    f = findfolder(dest, name)
    if f != None:
        print(f.name)
    else:
        f = createFolder(dest, test_sub)
        print("created", f.folderpath)


# folder_test()

In [8]:
def getsub(path):
    return path.replace(root.folderpath, "")


messages = src_folder.Items
print(messages.count)
counter = 0
while src_folder.Items.Count > 0:
    message = src_folder.Items.GetFirst()
    sub = getsub(message.Parent.FolderPath)
    f = findfolder(dest, sub)
    if f is None:
        f = createFolder(dest, sub)
    message.move(f)
    print("\r", counter, sub, message.senton, message.Subject, end="")
    counter += 1
#    if counter>10 :
#        break
print("done.")

159
 158 \#04 人事系 2021-02-12 19:52:19+00:00 【新卒採用】2022年新卒採用活動 公募一次グループディスカッション(以下GD)のご対応依頼done.（データ北海道）告について
